# Game prototype

In [1]:
from collections.abc import Callable
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List, Union, TypeVar, Tuple, Set, Type

import logging
import regex

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    filename="app.log",
    filemode="a",
)

I = TypeVar("I")  # placeholder for Image or emoji
#todo einen typen Canvas, der (image|text)* beschreibt (ist Canvas wirklich der richtige Name)
E = TypeVar("E", bound="Encoder")
D = TypeVar("D", bound="Decoder")

QueryType = Union[str, I, List[Union[str, I]]] #todo stattdessen Canvas
AlphabetType = Set[Union[str, I]] # todo: erlaube auch (optional), hier eine Freitextbeschreibung anzugeben.
#todo alphabet auf Canvas aufbauen

In [2]:
# ───────────────────────────────────
# EXAMPLE & TASK
# ───────────────────────────────────


@dataclass
class Example:
    """
    One example puzzle: contains the message to encode,
    the allowed alphabets for encoding, and the alphabet for decoding.
    """

    def __init__(
        self,
        message: QueryType,
        encoder_alphabet: QueryType,
        decoder_alphabet: QueryType,
    ):
        self.message = message
        self.encoder_alphabet = encoder_alphabet
        self.decoder_alphabet = decoder_alphabet


@dataclass()
class Task:
    """A set of examples forming a complete task."""

    def __init__(self, examples: List[Example]):
        self.examples = examples


@dataclass()
class RoundState:
    """
    Stores the intermediate and final results of a single example:
    the cipher produced by the sender and the decoded message from the receiver.
    Class used for logging
    """

    def __init__(self, example: Example):
        self.example = example
        self.cipher = None
        self.ciphers = []  # TODO: use
        self.message_prime = None
        self.message_primes = []  # TODO: use


# ───────────────────────────────────
# ENCODERS
# ───────────────────────────────────


class Encoder(ABC):
    """Abstract base for all encoders."""

    @abstractmethod
    def encode(
        self,
        cipher: QueryType,
        encoder_alphabet: AlphabetType,
    ) -> bool:
        """Check whether the cipher is valid given the alphabet."""
        pass


class WordsEncoder(Encoder, ABC):
    """Encoder allowing arbitrary combinations of symbols."""

    NAME = "Words Encoder"
    pass


class NWordsEncoder(Encoder, ABC):
    """Encoder restricting words to maximum length n."""

    NAME = "N Words Encoder"
    pass


class SingleChoiceEncoder(Encoder, ABC):
    """Encoder restricted to a single symbol."""

    NAME = "Single Choice Encoder"
    pass


class MultipleChoiceEncoder(Encoder, ABC):
    """Encoder restricted to a fixed number of symbols."""

    NAME = "Multiple Choice Encoder"
    pass


class PermutationsAllEncoder(Encoder, ABC):
    """Encoder restricted to permutation of the entire alphabet."""

    NAME = "Permutations All Encoder"
    pass


class PermutationsSubEncoder(Encoder, ABC):
    """Encoder allowing permutation of a subset of the alphabet."""

    NAME = "Permutations Sub Encoder"
    pass


# ───────────────────────────────────
# DECODERS
# ───────────────────────────────────


class Decoder(ABC):
    """Abstract base for all decoders."""

    @abstractmethod
    def decode(
        self,
        message_prime: QueryType,
        decoder_alphabet: AlphabetType,
    ) -> bool:
        """Check whether the decoded message is valid given the alphabet."""
        pass


class WordsDecoder(Decoder, ABC):
    """Decoder for arbitrary words over the alphabet."""

    NAME = "Words Decoder"
    pass


class NWordsDecoder(Decoder, ABC):
    """Decoder restricting words to maximum length n."""

    NAME = "WordsDecoder"
    pass


class SingleChoiceDecoder(Decoder, ABC):
    """Decoder restricted to a single symbol."""

    NAME = "N Words Decoder"
    pass


class MultipleChoiceDecoder(Decoder, ABC):
    """Decoder restricted to a fixed number of symbols."""

    NAME = "Multiple Choice Decoder"
    pass


class PermutationsAllDecoder(Decoder, ABC):
    """Decoder restricted to permutation of the entire alphabet."""

    NAME = "Permutations All Decoder"
    pass


class PermutationsSubDecoder(Decoder, ABC):
    """Decoder allowing permutation of a subset of the alphabet."""

    NAME = "Permutations Sub Decoder"
    pass


# ───────────────────────────────────
# VALIDATOR
# ───────────────────────────────────


class Validator(ABC):
    """Validates the correctness of the decoded message and awards points."""

    @abstractmethod
    def validate(self, round_state: RoundState) -> int:
        pass


# ───────────────────────────────────
# AGENTS
# ───────────────────────────────────


class Agent(ABC):
    """Interface for human or AI agent interacting with the game."""

    @abstractmethod
    def present_data(
        self,
        query: QueryType,
        alphabet: AlphabetType,
        name: str,
    ) -> None:
        """Display the query and alphabet to the agent."""
        pass

    @abstractmethod
    def get_data(self) -> QueryType:
        """Return the agent's response (cipher or decoded message)."""
        pass


class HumanAgent(Agent):
    """Agent interacting with a human user."""

    pass


class AIAgent(Agent):
    """Agent interacting with an AI model."""

    pass


# ───────────────────────────────────
# SENDER / RECEIVER
# ───────────────────────────────────


class Sender(ABC):
    """Handles encoding and communication with an agent."""

    def __init__(self, encoder_cls: Type[E], agent_cls: Type[Agent]):
        self.encoder = encoder_cls()
        self.agent = agent_cls()

    @abstractmethod
    def run_agent_encode(self, round_state: RoundState) -> QueryType:
        """Run the agent to produce a cipher."""
        pass

    @abstractmethod
    def verify_cipher(self, round_state: RoundState) -> bool:
        """Check if the produced cipher is valid."""
        pass


class Receiver(ABC):
    """Handles decoding and communication with an agent."""

    def __init__(self, decoder_cls: Type[D], agent_cls: Type[Agent]):
        self.decoder = decoder_cls()
        self.agent = agent_cls()

    @abstractmethod
    def run_agent_decode(self, round_state: RoundState) -> QueryType:
        """Run the agent to decode the cipher to message_prime."""
        pass

    @abstractmethod
    def verify_message_prime(self, round_state: RoundState) -> bool:
        """Check if the decoded message is valid."""
        pass


# ───────────────────────────────────
# GAME
# ───────────────────────────────────


class Game(ABC):
    """Game logic handling transmission, validation, and communication."""

    def __init__(
        self,
        task: Task,
        encoder_cls: Type[E],
        decoder_cls: Type[D], # TODO: potienzel unterschiedliche checker
        sender_cls: Type[Sender],
        receiver_cls: Type[Receiver],
        validator_cls: Type[Validator],
        agent_cls: Type[Agent],
        n_receivers: int,
    ):
        self.task = task

        self.sender = sender_cls(encoder_cls, agent_cls)

        self.receivers = [
            receiver_cls(decoder_cls, agent_cls) for _ in range(n_receivers)
        ]

        self.points = {r: 0 for r in self.receivers}

        self.validator = validator_cls()

    @abstractmethod
    def call_sender(self, round_state: RoundState) -> bool:
        """Calls sender, returns a tuple (cipher, is_valid)"""
        pass

    @abstractmethod
    def call_receivers(self, round_state: RoundState) -> bool:
        """Calls receiver(s) with cipher, returns a tuple (message_prime, is_valid)"""
        pass

    @abstractmethod
    def call_validator(self, round_state: RoundState) -> int:
        pass

    @abstractmethod
    def play_example(self, example: Example):
        pass

    @abstractmethod
    def play_task(self):
        """Play all examples in the task."""
        pass

In [3]:
# -----------------------------
# Encoder / Decoder
# -----------------------------

class MyWordsEncoder(WordsEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        return all(c in encoder_alphabet for c in regex.findall(r"\X", cipher))

# TODO: ask Nilas enc/dec to basicly the same

class MyWordsDecoder(WordsDecoder):
    def decode(self, message_prime, decoder_alphabet) -> bool:
        return all(c in decoder_alphabet for c in message_prime)


# -----------------------------
# Agent
# -----------------------------
import time
from IPython.display import clear_output
class HumanAgent(Agent):#todo soll von humanagent erben?? Warum heißt die Klasse hier auch HumanAgent? Das gibt es ja oben schon als abstract class?
    #todo: die task so umschreiben, dass sie einen format string bereitstellt (https://www.w3schools.com/python/ref_string_format.asp), der auch Informationen dazu gibt, was die Task ist (hier: gegeben ein *Filmtitel*, umschreibe ihn mit Emojis. Dabei darf {type} (-> jedes emoji beliebig oft benutzen))
    def present_data(self, query, alphabet, name):
        clear_output()
        print(f"\n{name} '{query}' using: {alphabet}")

    def get_data(self):
        query = input("\nEnter: ")
        print(f"\nYou typed: {query}")
        time.sleep(3)
        return query.strip().replace("\n", "").replace("\r", "") #todo: split. aber das Alphabet sollte vorgeben, ob nach jedem Character gesplittet oder bei einem bestimmten Trennzeichen -- und man muss das auch noch dem agent sagen. auch erlauben, einen index anzugeben? -> alle 3 über die Task lösen.


# -----------------------------
# Sender / Receiver / Validator
# -----------------------------
class MySender(Sender):

    def __init__(self, encoder_cls, agent_cls):
        super().__init__(encoder_cls, agent_cls)

    def run_agent_encode(self, round_state):
        self.agent.present_data(
            round_state.example.message,
            round_state.example.encoder_alphabet,
            type(self.encoder).NAME,
        )
        return self.agent.get_data()

    def verify_cipher(self, round_state):
        return self.encoder.encode(
            round_state.cipher, round_state.example.encoder_alphabet
        )


class MyReceiver(Receiver):

    def __init__(self, decoder_cls, agent_cls):
        super().__init__(decoder_cls, agent_cls)

    def run_agent_decode(self, round_state):
        self.agent.present_data(
            round_state.cipher,
            round_state.example.decoder_alphabet,
            type(self.decoder).NAME,
        )
        return self.agent.get_data()

    def verify_message_prime(self, round_state):
        return self.decoder.decode(
            round_state.message_prime, round_state.example.decoder_alphabet
        )


class MyValidator:
    def validate(self, round_state):
        return int(
            round_state.example.message.lower() == round_state.message_prime.lower()
        )


# -----------------------------
# Game
# -----------------------------


class MyGame(Game):

    def __init__(
        self,
        task,
        encoder_cls,
        decoder_cls,
        sender_cls,
        receiver_cls,
        validator_cls,
        agent_cls,
        n_receivers,
    ):
        super().__init__(
            task,
            encoder_cls,
            decoder_cls,
            sender_cls,
            receiver_cls,
            validator_cls,
            agent_cls,
            n_receivers,
        )

        # For simplicity, assume only one receiver # TODO: remove
        self.receiver = self.receivers[0] if self.receivers else None

    def call_sender(self, round_state):
        cipher = self.sender.run_agent_encode(round_state)
        round_state.cipher = cipher
        success = self.sender.verify_cipher(round_state)
        return success

    def call_receivers(self, round_state):
        message_prime = self.receiver.run_agent_decode(round_state)
        round_state.message_prime = message_prime
        success = self.receiver.verify_message_prime(round_state)
        return success

    def call_validator(self, round_state):
        return self.validator.validate(round_state)

    def play_example(self, example):

        round_state = RoundState(example)

        success = self.call_sender(round_state)
        while not success:
            print("\nRetry")
            success = self.call_sender(round_state)

        print("\nNext Phase")

        success = self.call_receivers(round_state)
        while not success:
            print("\nRetry")
            success = self.call_receivers(round_state)

        print("\nNext Round")

        points = self.call_validator(round_state)
        self.points[self.receiver] += points

        print(f"\nreceived points: {points}")

    def play_task(self):
        for example in self.task.examples:
            self.play_example(example)

        print(f"\ntotal points: {self.points[self.receiver]}")

In [ ]:
if __name__ == "__main__":
    emoji_alphabet = {"❄️", "🛳️", "❤️", "💥", "🌊", "🧊", "💀", "🦈", "🚢", "🌹"}
    letter_alphabet = set("abcdefghijklmnopqrstuvwxyz")

    examples = [
        Example("Titanic", emoji_alphabet, letter_alphabet),
        Example("Inception", emoji_alphabet, letter_alphabet),
        Example("Avatar", emoji_alphabet, letter_alphabet),
    ]
    task = Task(examples)

    game = MyGame(
        task=task,
        encoder_cls=MyWordsEncoder,
        decoder_cls=MyWordsDecoder,
        sender_cls=MySender,
        receiver_cls=MyReceiver,
        validator_cls=MyValidator,
        agent_cls=HumanAgent,
        n_receivers=1,
    )

    game.play_task()


Words Decoder '🚢🧊💀' using: {'m', 'g', 'a', 'e', 'q', 'h', 'j', 'p', 'o', 'y', 'd', 's', 'n', 'u', 'b', 'z', 'c', 'f', 't', 'l', 'w', 'k', 'v', 'i', 'r', 'x'}


In [6]:
# ──────────────────────────────
# N Words Encoder / Decoder
# ──────────────────────────────


class MyWordsEncoder(WordsEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols)
 

class MyWordsDecoder(WordsDecoder):
    def decode(self, message_prime, decoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", message_prime)
        return all(c in decoder_alphabet for c in symbols)


# ──────────────────────────────
# N Words Encoder / Decoder
# ──────────────────────────────


class MyNWordsEncoder(NWordsEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols) and len(symbols) <= n


class MyNWordsDecoder(NWordsDecoder):
    def decode(self, message_prime, decoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", message_prime)
        return all(c in decoder_alphabet for c in symbols) and len(symbols) <= n


# -----------------------------
# SingleChoice
# -----------------------------
class MySingleChoiceEncoder(
    SingleChoiceEncoder
):  # TODO ask Niklas if same as nword with n=n 1
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols) and len(symbols) == 1


class MySingleChoiceDecoder(SingleChoiceDecoder):
    def decode(self, message_prime, decoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", message_prime)
        return all(c in decoder_alphabet for c in symbols) and len(symbols) == 1


# -----------------------------
# MultipleChoice
# -----------------------------
class MyMultipleChoiceEncoder(MultipleChoiceEncoder):  # TODO ask Niklas if same as word
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols)


class MyMultipleChoiceDecoder(MultipleChoiceDecoder):
    def decode(self, message_prime, decoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", message_prime)
        return all(c in decoder_alphabet for c in symbols)


# -----------------------------
# PermutationsAll
# -----------------------------
class MyPermutationsAllEncoder(PermutationsAllEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return sorted(symbols) == sorted(encoder_alphabet)


class MyPermutationsAllDecoder(PermutationsAllDecoder):
    def decode(self, message_prime, decoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", message_prime)
        return sorted(symbols) == sorted(decoder_alphabet)


# -----------------------------
# PermutationsSub
# -----------------------------
class MyPermutationsSubEncoder(
    PermutationsSubEncoder
):  # TODO ask Niklas if same as word
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols) and len(symbols) <= len(
            encoder_alphabet
        )


class MyPermutationsSubDecoder(PermutationsSubDecoder):
    def decode(self, message_prime, decoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", message_prime)
        return all(c in decoder_alphabet for c in symbols) and len(symbols) <= len(
            decoder_alphabet
        )